<a href="https://colab.research.google.com/github/devdunie95/Apache-Struts-2.5-2.5.12---REST-Plugin-XStream-Remote-Code-Execution/blob/master/Phishing_URL_FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##  Phishing Website Detection - Feature Extraction

###### Data preprocessing

######  Pre-Processing the data before building a model and also Extracting the features from the data based on certain conditions

### Showing Results
###### 0 --- indicates Legitimate
###### 1 --- indicates Phishing
###### 2 --- indicates Suspicious



In [ ]:
!pip install python-whois

     |████████████████████████████████| 92kB 6.9MB/s 
  Created wheel for python-whois: filename=python_whois-0.7.3-cp36-none-any.whl size=87704 sha256=ef7cb77dbddae4dce240feb41f4d2818a7f1d0f959394ccaa8f3afe965d54f7b
  Stored in directory: /root/.cache/pip/wheels/12/3c/9b/901b7deea1fa960f9abfd84df82414dff215c4a1d6869bcea2
Successfully built python-whois


In [ ]:
# importing packages
import pandas as pd
import numpy as np
from urllib.parse import urlparse,urlencode
import re 
import googlesearch
from bs4 import BeautifulSoup
import requests
import whois
import urllib.request
from datetime import datetime
import time
import socket
from urllib.error import HTTPError
from cython.parallel import prange
from numba import prange

In [ ]:
from datetime import datetime
itime = datetime.now()
for i in prange(0,10000):
    pass
ftime = datetime.now()
print(ftime-itime)

0:00:00.000456


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving phishing.csv to phishing.csv
Saving top 1m.csv to top 1m.csv


In [ ]:
computer_raw_data=pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/top 1m.csv",header=None,names=['urls'])

In [ ]:
computer_raw_data=pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/phishing.csv",header=None,engine='python')
#computer_raw_data/content/drive/My Drive/Colab Notebooks/data/1000-phishing.txt 

In [ ]:
cd = None
class FeatureExtraction:
    def __init__(self):
        pass
    
    def getProtocol(self,url):
        return urlparse(url).scheme
    
    def getDomain(self,url):
        return urlparse(url).netloc
    
    def getPath(self,url):
        return urlparse(url).path


        # feature - 1
    
    def havingIP(self,url):
        # If an IP address is used as an alternative of the domain name in the URL, such as “http://125.98.3.123/fake.html”,users can be sure that someone is trying to steal their personal information. Sometimes, the IP address is even transformed into hexadecimal code
        # If The Domain Part has an IP Address → Phishing
        # Otherwise→ Legitimate
       
        match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
        if match:
            #print match.group()
            return 1            # phishing
        else:
            #print 'No matching pattern found'
            return 0            # legitimate


        # feature - 2
    
    def long_url(self,url):
        # If the length of the URL is greater than or equal 54 characters then the URL classified as phishing
        
        if len(url) < 54:
            return 0            # legitimate
        elif len(url) >= 54 and len(url) <= 75:
            return 2            # suspicious
        else:
            return 1            # phishing
    

        # feature - 3

    def have_at_symbol(self,url):
        # URL’s having “@” Symbol
        # This function is used to check whether the URL contains @ symbol or not
        # Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.
        
        if "@" in url:
            return 1            # phishing
        else:
            return 0            # legitimate
    

        # feature - 4

    def redirection(self,url):
        # If the url has symbol(//) after protocol then such URL is to be classified as phishing
        # The existence of “//” within the URL path means that the user will be redirected to another website.An example of such URL’s is: “http://www.legitimate.com//http://www.phishing.com”. 
        # We examine the location where the “//” appears. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. 
        # However, if the URL employs “HTTPS” then the “//” should appear in the seventh position.
        
        if "//" in urlparse(url).path:
            return 1            # phishing
        else:
            return 0            # legitimate
        

        # feature - 5

    def prefix_suffix_separation(self,url):
        # If the domain has '-' symbol then it is considered as phishing site
        # The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name.so that users feel that they are dealing with a legitimate webpage. 
        # For example http://www.Confirme-paypal.com/.
        
        if "-" in urlparse(url).netloc:
            return 1            # phishing
        else:
            return 0            # legitimate
        

        # feature - 6

    def sub_domains(self,url):
        # Sub-Domain and Multi Sub-Domains
        # The legitimate URL link has two dots in the URL since we can ignore typing “www.”. 
        # If the number of dots is equal to three then the URL is classified as “Suspicious” since it has one sub-domain.
        # However, if the dots are greater than three it is classified as “Phishy” since it will have multiple sub-domains

        if url.count(".") < 3:
            return 0            # legitimate
        elif url.count(".") == 3:
            return 2            # suspicious
        else:
            return 1            # phishing
        

        # feature - 7

    def shortening_service(self,url):
        # Using URL Shortening Services “TinyURL”
        # Tiny URL -> phishing otherwise legitimate
        # we have imported re module from the begining. 

        match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
        if match:
            return 1               # phishing
        else:
            return 0               # legitimate


        # feature - 8

        """This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit.
However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database 
(Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios,
legitimate websites ranked among the top 100,000. Furthermore,
if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.
Otherwise, it is classified as “Suspicious”.""" 

    def web_traffic(self,url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        except TypeError:
            return 1
        except HTTPError:
            return 2
        rank= int(rank)
        if (rank<100000):
            return 0
        else:
            return 2
        
        
        # feature - 9

    def domain_registration_length(self,url):
        # Phishing website lives for a short period of time, and trustworthy domains are regularly paid for several years in advance.
        
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1      # phishing
        else:
            expiration_date = domain_name.expiration_date
            today = time.strftime('%Y-%m-%d')
            today = datetime.strptime(today, '%Y-%m-%d')
            if expiration_date is None:
                return 1
            elif type(expiration_date) is list or type(today) is list :
                return 2     #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
            else:
                creation_date = domain_name.creation_date
                expiration_date = domain_name.expiration_date
                if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                    try:
                        creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                    except:
                        return 2
                registration_length = abs((expiration_date - today).days)
                if registration_length / 365 <= 1:
                    return 1 # phishing
                else:
                    return 0 # legitimate


        # feature - 10

    def age_domain(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            creation_date = domain_name.creation_date
            expiration_date = domain_name.expiration_date
            if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                try:
                    creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                    expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                except:
                    return 2
            if ((expiration_date is None) or (creation_date is None)):
                return 1
            elif ((type(expiration_date) is list) or (type(creation_date) is list)):
                return 2
            else:
                ageofdomain = abs((expiration_date - creation_date).days)
                if ((ageofdomain/30) < 6):
                    return 1
                else:
                    return 0
     
        # feature - 11
    def dns_record(self,url):
        """For phishing websites, either the claimed identity is not recognized by the WHOIS database (Whois 2005) or 
no records founded for the hostname (Pan and Ding 2006). 
If the DNS record is empty or not found then the website is classified as “Phishing”, otherwise it is classified as “Legitimate”."""
        
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
            #rint(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            return 0
        
   
    def statistical_report(self,url):
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
            y = h[0][1]
            hostname = hostname[y:]
            h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
            z = int(len(h))
            if z != 0:
                hostname = hostname[:h[0][0]]
        url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
        try:
            ip_address = socket.gethostbyname(hostname)
            ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
        except:
            return 1

        if url_match:
            return 1
        else:
            return 0
        
        
        # feature - 12

    def https_token(self,url):
      # The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users. For example, http://https-www-paypal-it-webapps-mpp-home.soft-hair.com/.

        match=re.search('https://|http://',url)
        try:
            if match.start(0)==0 and match.start(0) is not None:
                url=url[match.end(0):]
                match=re.search('http|https',url)
                if match:
                    return 1
                else:
                    return 0
        except:
            return 1

In [ ]:
#features
protocol = []
domain = []
path = []
having_ip = []
len_url = []
having_at_symbol = []
redirection_symbol = []
prefix_suffix_separation = []
sub_domains = []
tiny_url = []
abnormal_url = []
web_traffic = []
domain_registration_length = []
dns_record = []
statistical_report = []
age_domain = []
http_tokens = []


In [ ]:
# object creation

fe = FeatureExtraction()
rows = len(computer_raw_data["urls"])

for i in range(0,rows):
    url=computer_raw_data["urls"][i]
    print(i ),print(url) 
    protocol.append(fe.getProtocol(url))
    path.append(fe.getPath(url))
    domain.append(fe.getDomain(url))
    having_ip.append(fe.havingIP(url))
    len_url.append(fe.long_url(url))
    having_at_symbol.append(fe.have_at_symbol(url))
    redirection_symbol.append(fe.redirection(url))
    prefix_suffix_separation.append(fe.prefix_suffix_separation(url))
    sub_domains.append(fe.sub_domains(url))
    tiny_url.append(fe.shortening_service(url))
    #web_traffic.append(fe.web_traffic(url))
    domain_registration_length.append(fe.domain_registration_length(url))
    dns_record.append(fe.dns_record(url))
    statistical_report.append(fe.statistical_report(url))
    age_domain.append(fe.age_domain(url))
    http_tokens.append(fe.https_token(url))
    

KeyError: ignored

In [ ]:
label = []
for i in range(0, rows):
    label.append(1)

In [ ]:
import pandas as pd

d={'Protocol':pd.Series(protocol),'Domain':pd.Series(domain),'Path':pd.Series(path),'Having_IP':pd.Series(having_ip),
   'URL_Length':pd.Series(len_url),'Having_@_symbol':pd.Series(having_at_symbol),
   'Redirection_//_symbol':pd.Series(redirection_symbol),'Prefix_suffix_separation':pd.Series(prefix_suffix_separation),
   'Sub_domains':pd.Series(sub_domains),'tiny_url':pd.Series(tiny_url),'web_traffic' : pd.Series(web_traffic) ,
   'domain_registration_length':pd.Series(domain_registration_length),'dns_record':pd.Series(dns_record),
   'statistical_report':pd.Series(statistical_report),'age_domain':pd.Series(age_domain),'http_tokens':pd.Series(http_tokens),
   'label':pd.Series(label)}

data = pd.DataFrame(d)
data

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,label
0,http,v2.email-marketing.adminsimple.com,/track/link,0,1,0,0,1,1,0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1,http,bid.openx.net,/json,0,1,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,http,webmail2.centurytel.net,/hwebmail/services/go.php,1,1,0,0,0,1,0,1.0,1.0,0.0,0.0,2.0,1.0,1.0
3,http,www.google.com.ng,/imgres,0,1,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,1.0,1.0
4,http,webmail2.centurytel.net,/hwebmail/services/go.php,0,1,0,0,0,1,0,1.0,1.0,0.0,0.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11105,http,highedgesolar.com,/nw/includes/,0,0,0,0,0,0,0,NaN,1.0,1.0,0.0,1.0,0.0,NaN
11106,http,jimfangimporters.yolasite.com,/,0,0,0,0,0,0,0,NaN,1.0,0.0,0.0,0.0,0.0,NaN
11107,http,liuheng.chengxuren.com,/Images/,0,0,0,0,0,0,0,NaN,1.0,1.0,1.0,1.0,0.0,NaN
11108,http,mcnaotempreco.net,/site/portal/,0,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
data.to_csv("phishing-urls.csv",index=False,encoding='UTF-8')

NameError: ignored

In [ ]:
data.to_csv('phishing-urls.csv',na_rep='Unkown') # missing value save as Unknown

In [ ]:
data.to_csv("legitimate-urls.csv",index=False,encoding='UTF-8')


data.to_csv("phishing-urls.csv",index=False,encoding='UTF-8')
